In [ ]:
'''
1. 使用 pandas 读取数据文件为 DataFrame
首先我们需要在机器上安装 pandas，如果在安装 TensorFlow 的过程中没有自动安装 pandas ，
那么我们就需要手动安装 pandas ：pip install pandas

对于CSV文件，我们只需要使用 pd.read_csv() 函数就可以正确地读取CSV文件中的数据为 DataFrame 格式数据。
'''
import pandas as pd
import tensorflow as tf

train_file = tf.keras.utils.get_file("train.csv", "https://storage.googleapis.com/tf-datasets/titanic/train.csv")

train_df = pd.read_csv(train_file)
# 显示数据集合的前 5 条数据
train_df.head()

'''
2. 对 DataFrame 进一步处理2. 对 DataFrame 进一步处理
在这一个步骤之中，我们要进行的工作主要有：
a.离散数据离散化，并且进行编码；
b.分离标签与数据。
对于离散数据离散化，我们可以使用 pandas 内置的 pd.Categorical() 方法进行；
而后我们需要进行的是离散数据的编码，此时我们可以通过 DataFrame.xxx.cat.codes 来实现，
其中 xxx 为 DataFrame 的属性。
'''
train_df['sex'] = pd.Categorical(train_df['sex'])
train_df['sex'] = train_df.sex.cat.codes

train_df['deck'] = pd.Categorical(train_df['deck'])
train_df['deck'] = train_df.deck.cat.codes

train_df['embark_town'] = pd.Categorical(train_df['embark_town'])
train_df['embark_town'] = train_df.embark_town.cat.codes

train_df['alone'] = pd.Categorical(train_df['alone'])
train_df['alone'] = train_df.alone.cat.codes

train_df['classes'] = pd.Categorical(train_df['class'])
train_df['classes'] = train_df.classes.cat.codes
# 要分离标签与数据，我们可以直接通过 pop() 函数来实现
train_df.pop('class')
labels = train_df.pop('survived')

train_df.head()

'''
3. 构建 tf.data.Dataset 数据集
在这一步，我们可以使用之前的方法 tf.data.Dataset.from_tensor_slices 来进行数据集的构造。

对于该示例，我们可以通过以下代码实现：
'''
train_dataset = tf.data.Dataset.from_tensor_slices((train_df.values, labels.values))
train_dataset = train_dataset.shuffle(len(train_df)).batch(32)  # 打乱

'''
4. 数据集的使用
在这里，我们使用和之前相似的模型进行演示，于是我们首先定义一个分类器，然后进行训练。
'''
model = tf.keras.Sequential([
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dense(64, activation='relu'),
  tf.keras.layers.Dense(1, activation='sigmoid'),
])

model.compile(
    loss='binary_crossentropy',
    optimizer='adam',
    metrics=['accuracy'])

model.fit(train_dataset, epochs=30)

